In [1]:
import stim
import numpy as np
import qiskit
import itertools
import nbimporter

from NonCliff_1Meas_V4 import *
from stab_rank_approx import *

from qiskit.quantum_info import Pauli, PauliList, StabilizerState
from typing import List, Tuple

In [208]:
def find_nearest_neighbors_general_V2(num_qubits: int, stab_state: np.array) -> List:
    """ 
    Construct list of all nearest neighbor stabilizer states for arbitrary stabilizer state.

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find neighbors

    Returns:
    --------
    List of nearest neighbors
    """
    bias = 1/np.sqrt(2)
    all_paulis = generate_all_paulis(num_qubits)
    new_stabs = [p.to_matrix() @ stab_state for p in all_paulis]
    new_stabs_orthogonal = [stab for stab in new_stabs if np.vdot(stab_state, stab) == 0]
    neighbors = [np.add(stab_state, stab) * bias for stab in new_stabs_orthogonal]
    neighbors_dict = {stab.tobytes(): stab for stab in neighbors}
    # return list(neighbors_dict.values())
    final_neighbors = list(neighbors_dict.values())
    final_neighbors.append(stab_state)
    return final_neighbors

In [272]:
def is_orthogonal(used_states: List, curr_stab: np.array):
    """
    Checks that all the stabilizer states that are being used for decomposition are indeed orthogonal

    Params:
    -------
    used_states - stabilizer states used so far 
    curr_stab - Next stabilizer state to be used for decomposition
    """ 
    if (np.array_equal(curr_stab, None)):
        return False
    for elem in used_states:
        if (np.vdot(curr_stab, elem) != 0):
            return False 
    return True 

def iterated_optimal_nearest_neighbor_V2(num_qubits:int, eig_vec:np.array, init_stab_state: np.array, used_states: List) -> np.array:
    """ 
    Iteratively compute the optimal nearest neighbor whose inner product with 'eig_vec' is maximal

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate
    init_stab_state - Initial stabilizer state where we start our search from 

    Returns:
    --------
    Optimal single state that best approximates 'eig_vec'
    """
    num_iterations = 2 ** num_qubits
    curr_stab_state = init_stab_state
    max_inner_prod = 0
    curr_best_stab_state = None
    for _ in range(num_iterations):
        curr_neighbors = find_nearest_neighbors_general_V2(num_qubits, curr_stab_state)
        if (used_states != []):
            cp = []
            for neighbor in curr_neighbors:
                for state in used_states:
                    if (np.vdot(neighbor, state) == 0):
                        cp.append(neighbor)
            curr_neighbors = cp
        optimal_neighbor = curr_neighbors[np.argmax([np.abs(np.vdot(neighbor, eig_vec)) for neighbor in curr_neighbors])]
        curr_stab_state = optimal_neighbor 
        #curr_inner_prod = np.vdot(curr_stab_state, eig_vec)
        curr_inner_prod = np.abs(np.vdot(curr_stab_state, eig_vec))
        if (np.array_equal(curr_best_stab_state,curr_stab_state)):
            break
        if (curr_inner_prod > max_inner_prod):
            max_inner_prod = curr_inner_prod 
            curr_best_stab_state = curr_stab_state

    return curr_best_stab_state, max_inner_prod

In [250]:
def one_state_stab_approx_V2(num_qubits:int, eig_vec: np.array, used_states: List) -> np.array:
    """ 
    Find best (?) single stabilizer state approximation for eigen vector 

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate

    Returns:
    --------
    Best (?) single stabilizer state that approximates eigenvector
    """

    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    #final_approx_state = np.zeros((2**num_qubits,), dtype=complex) # Where result for best stab state approx is stored
    
    # Find basis state with maximum coefficient
    max_coeff = state_max_coeff(eig_vec, basis_vecs)
    init_stab_state = basis_vecs[max_coeff]

    # Compute optimal nearest stab state in iterated fashion
    final_approx_state, max_inner_prod = iterated_optimal_nearest_neighbor_V2(num_qubits, eig_vec, init_stab_state, used_states)

    return final_approx_state, max_inner_prod

In [359]:
def stab_rank_approx_four_V2(num_qubits: int, eig_vec: np.array) -> Tuple:
    """ 
    Approximate the stabilizer rank (using approach discussed)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0 
    approx_vec = np.zeros(len(min_eigvec), dtype=complex) # Vector we wish to construct to approximate 'eig_vec'
    prec_param = 0.02 # Precision parameter to see how close dot prod of 'approx_vec' and 'eig_vec' is
    used_states = []
    norm_factor = 0
    last_inner = 0
    is_approx = False # While loop termination condition

    copy_vec = eig_vec.copy()
    while (is_approx == False):
        # Calculate nearest stabilizer state
        s, s_inner_prod = one_state_stab_approx_V2(num_qubits, eig_vec, used_states)
        #print("Is orthogonal?: " + str(is_orthogonal(used_states, s)))
        used_states.append(s)
        #print("Stab: " + str(list(s)))

        # Project 'eig_vec' on s
        s_norm = np.linalg.norm(s)
        s_coeff = np.vdot(s, eig_vec)
        if (norm_factor == 0):
            proj_vec = (s_coeff/s_norm**2) * s
        else:
            proj_vec = norm_factor * (s_coeff/s_norm**2) * s

        # Subtract projection of 'eig_vec' on s from 'eig_vec'
        unnorm_eig_vec = eig_vec - proj_vec 
        norm_factor = np.linalg.norm(unnorm_eig_vec)
        eig_vec = unnorm_eig_vec / norm_factor

        # Add projection to approximation and update stab rank
        approx_vec += proj_vec
        approx_stab_rank += 1

        # Check closeness of approximation to original 'eig_vec'

        """ 
        if (last_inner > np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)):
            approx_vec -= proj_vec
            approx_stab_rank -= 1
            break
        """

        last_inner = np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)
        #print("How close: " + str(last_inner))
        if (np.abs(1 - np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)) <= prec_param):
            is_approx = True 
        #print(np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec))
        #if (np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec) <= prec_param):
        #    is_approx = True

        if (approx_stab_rank == 2 ** num_qubits):
            break

        #print("Approx vec so far: " + str(list(approx_vec)))
        
    approx_vec = approx_vec / np.linalg.norm(approx_vec)
    
    return approx_stab_rank, approx_vec


In [363]:
num_wit_qubits = 25 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 3 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

num_layers = 30

assert(num_layers > 0)

In [357]:
p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
succinct_p_sum = p_sum.succinct_p_sum(total_qubits)
min_eigvec = succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))[1]
new_total_qubits = len(succinct_p_sum.get_sum()[0].get_pauli_str(succinct_p_sum.get_sum()[0].get_pauli()))
print(succinct_p_sum)
print(list(min_eigvec))

[(1+0j) Z]
[0j, (1+0j)]


In [355]:
approx_stab_rank, approx_vec = stab_rank_approx_four_V2(new_total_qubits, min_eigvec)
print(approx_stab_rank, approx_vec)

Is orthogonal?: True
Stab: [(1+0j), 0j]
How close: (1+0j)
Approx vec so far: [(1+0j), 0j]
1 [1.+0.j 0.+0.j]


/var/folders/5d/012l939x26zc05mjshslzh2h0000gn/T/ipykernel_66451/2307204624.py:41: RuntimeWarning: invalid value encountered in true_divide
  eig_vec = unnorm_eig_vec / norm_factor


In [367]:
num_iter = 100
stab_rank_avg = 0
num_t_gates = 0
t_gate_cap = 7
stab_rank_list = []
for _ in range(t_gate_cap):
    stab_rank_avg = 0
    for i in range(num_iter):
        p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
        succinct_p_sum = p_sum.succinct_p_sum(total_qubits)
        min_eigvec = succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))[1]
        new_total_qubits = len(succinct_p_sum.get_sum()[0].get_pauli_str(succinct_p_sum.get_sum()[0].get_pauli()))
        approx_stab_rank, _ = stab_rank_approx_four_V2(new_total_qubits, min_eigvec)
        stab_rank_avg += approx_stab_rank
    stab_rank_avg = stab_rank_avg / num_iter
    print(stab_rank_avg)
    stab_rank_list.append((num_t_gates, stab_rank_avg))
    num_t_gates += 1

/var/folders/5d/012l939x26zc05mjshslzh2h0000gn/T/ipykernel_66451/3677357341.py:41: RuntimeWarning: invalid value encountered in true_divide
  eig_vec = unnorm_eig_vec / norm_factor


1.0
1.45
2.36
3.97
5.69


KeyboardInterrupt: 